In [ ]:
#!pip install streamlit pyngrok langchain chromadb google-generativeai sentence-transformers --quiet
#!pip install -U langchain-community
#!pip install langchain-google-genai
#!pip install streamlit
#!pip install pyngrok

In [ ]:
#!pip install --upgrade google-genai

In [ ]:
import streamlit as st
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
import os

# Set API key
os.environ['GOOGLE_API_KEY'] = ""

# Sample questions dataset
questions_data = [
    {"role": "Data Scientist", "question": "Explain the difference between supervised and unsupervised learning."},
    {"role": "Data Scientist", "question": "What is overfitting and how do you prevent it?"},
    {"role": "ML Engineer", "question": "Explain gradient descent."},
    {"role": "ML Engineer", "question": "How do you optimize a deep learning model?"}
]

# Create embeddings and vector store
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_store = Chroma.from_texts(
    [q["question"] for q in questions_data],
    embedding=embedding_model,
    metadatas=[{"role": q["role"]} for q in questions_data]
)

def retrieve_interview_question(role):
    results = vector_store.similarity_search(role, k=1)
    return results[0].page_content if results else None

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7)

def evaluate_answer_and_suggest_correct(question, user_answer):
    template = """
    Interview Question: {question}
    Candidate's Answer: {user_answer}

    Please do the following:
    1. Evaluate the candidate's answer in detail.
    2. Suggest improvements if any.
    3. If the answer is incorrect or incomplete or NA, provide the correct model answer separately.

    Format your response like this:
    Feedback: <your feedback>
    Correct Answer (if applicable): <correct answer or 'N/A'>
    """

    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm
    response = chain.invoke({"question": question, "user_answer": user_answer})

    return response.content

# ---------------------------- Streamlit Interface ----------------------------

st.title("🧠 AI-Powered Interview Assistant")

role = st.selectbox("Select Role:", ["Data Scientist", "ML Engineer"])

if st.button("Get Interview Question"):
    question = retrieve_interview_question(role)
    if question:
        st.subheader("Interview Question:")
        st.write(question)

        user_answer = st.text_area("Your Answer:")

        if st.button("Evaluate Answer"):
            if user_answer.strip():
                feedback = evaluate_answer_and_suggest_correct(question, user_answer)
                st.subheader("AI Feedback:")
                st.write(feedback)
            else:
                st.warning("Please enter your answer before clicking 'Evaluate Answer'.")
    else:
        st.error("No question found for this role in the dataset.")


In [ ]:
#!pip install streamlit pyngrok langchain langchain-community langchain-google-genai chromadb google-genai

from pyngrok import ngrok
import os

# Set your ngrok authtoken (replace with your actual token)
ngrok.set_auth_token("")

# Kill any existing tunnels
ngrok.kill()

# Start the tunnel
public_url = ngrok.connect(port=8501)
print(f"🌐 Public URL: {public_url}")

# Run Streamlit app
!streamlit run interview_assistant.py --server.port 8501
